# Multi-threading in Python

Demonstrates basic functionality of Python 
[threading](https://docs.python.org/3.5/library/threading.html) 
module in Python3.x to perform parallel operations on a single node (following
the SMP paradigm). 

# %load "../batch_job.slurm"

```
#!/bin/bash

#SBATCH --nodes=1
#SBATCH --tasks-per-node=1
#SBATCH --cpus-per-task=8
#SBATCH --time=0-00:04:00
#SBATCH --mem=999M

# Sources the appropriate packages and conda environments
source source_file.sh 

echo $SLURM_JOB_NODELIST

# Runs the program 
./compute_pi.py 
date
```

# A practical(?) example

From [Rosetta Code](http://rosettacode.org/wiki/Parallel_Brute_Force#Python)

> Task

> Find, through brute force, the five-letter passwords corresponding with the following SHA-256 hashes:

> 1. 1115dd800feaacefdf481f1f9070374a2a81e27880f187396db67958b207cbad
> 2. 3a7bd3e2360a3d29eea436fcfb7e44c735d117c42d1c1835420b6b9942dd4f1b
> 3. 74e1bb62f8dabb8125a58852b63bdf6eaef667cb56ac7f7cdba6d7305c50a22f

> Your program should naively iterate through all possible passwords consisting only of five lower-case ASCII English letters. It should use concurrent or parallel processing, if your language supports that feature. You may calculate SHA-256 hashes by calling a library or through a custom implementation. Print each matching password, along with its SHA-256 hash.

## The worker function

The total number of passwords we need to try is $26^5$, so we'll pass each integer from $0$ to $26^5$ and map it to it's corresponding word and hash. Essentially, what we need to do is convert each base-10 integer to base-26; to get the first letter of the word, we divide by $26^4$ to get a value in the range $0-25$. Then, we take the remainder and divide by $26^3$ to get the second letter and so on. To get the ASCII value for each letter, we have to add $97$. Finally, we compute the sha256 digest.

In [36]:
from hashlib import sha256

hashes = {
    "1115dd800feaacefdf481f1f9070374a2a81e27880f187396db67958b207cbad",
    "3a7bd3e2360a3d29eea436fcfb7e44c735d117c42d1c1835420b6b9942dd4f1b",
    "74e1bb62f8dabb8125a58852b63bdf6eaef667cb56ac7f7cdba6d7305c50a22f"
}

def hash_from_serial(serial):
    divisor = 456976
    letters = []
    for i in range(5):
        letter, serial = divmod(serial, divisor)
        letters.append( 97 + int(letter) )
        divisor /= 26
    digest = sha256(bytes(letters)).hexdigest()
    if digest in hashes:
        password = "".join(chr(x) for x in letters)
        return {digest: password}

Since we don't have $26^5$ threads to work with, we need to divide the work up into chunks and assign each chunk to a worker.

In [35]:
def loop_hash(start, span, data):
    for serial in range(start, start + span):
        r = hash_from_serial(serial)
        if r is not None:
            data.update(r)

In [19]:
import os
from math import ceil

num_passwords = int(26 ** 5)
num_threads = int(os.environ['SLURM_CPUS_PER_TASK'])
chunksize = ceil(num_passwords / num_threads)

assert( chunksize * num_threads >= num_passwords)

In [13]:
def string_to_serial(s):
    return sum((int(ord(c)) - 97) * 26 ** i for i, c in enumerate(reversed(s)))

serial = string_to_serial('zyzzx')
r = hash_from_serial(serial)
print(serial, r)

11863797 {'1115dd800feaacefdf481f1f9070374a2a81e27880f187396db67958b207cbad': 'zyzzx'}


In [16]:
d = {}
loop_hash(11863797, 2, d)
print(d)

{'1115dd800feaacefdf481f1f9070374a2a81e27880f187396db67958b207cbad': 'zyzzx'}


In [20]:
import threading

def use_threading():
    threads = []
    serial_start = 0

    # Here, the dict will be mutated by the worker function
    pwords = dict()

    # Starts the processes
    for i in range(num_threads):
        t = threading.Thread(target=loop_hash, args=(serial_start, chunksize, pwords))
        serial_start += chunksize
        threads.append(t)
        t.start()

    # Joins the worker threads to the "master" thread
    for i in range(num_threads):
        t.join()

    print(pwords)
    
use_threading()

{'3a7bd3e2360a3d29eea436fcfb7e44c735d117c42d1c1835420b6b9942dd4f1b': 'apple', '74e1bb62f8dabb8125a58852b63bdf6eaef667cb56ac7f7cdba6d7305c50a22f': 'mmmmm', '1115dd800feaacefdf481f1f9070374a2a81e27880f187396db67958b207cbad': 'zyzzx'}


In [37]:
%timeit -n 1 -r 1 use_threading()

{'h2': 'apple', 'h3': 'mmmmm', 'h1': 'zyzzx'}
1 loop, best of 1: 53.1 s per loop


The `multiprocessing` module takes care of chunking for us if we pass it the chunk size. 

In [32]:
from multiprocessing.pool import ThreadPool

def use_thread_pool():
    p = ThreadPool(num_threads)
    pwords = dict()
    for v in filter(lambda x: x is not None, 
                    p.map(hash_from_serial, (i for i in range(num_passwords)), chunksize=chunksize)):
        pwords.update(v)
    print(pwords)


We can see that this also takes about a minute to execute.

In [31]:
%timeit -n 1 -r 1 use_thread_pool()

{'3a7bd3e2360a3d29eea436fcfb7e44c735d117c42d1c1835420b6b9942dd4f1b': 'apple', '74e1bb62f8dabb8125a58852b63bdf6eaef667cb56ac7f7cdba6d7305c50a22f': 'mmmmm', '1115dd800feaacefdf481f1f9070374a2a81e27880f187396db67958b207cbad': 'zyzzx'}
1 loop, best of 1: 1min 2s per loop


## Note that `ThreadPool` is not recommended!

As per the [Python documentation](https://docs.python.org/3.6/library/multiprocessing.html)

>The multiprocessing package offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads. Due to this, the multiprocessing module allows the programmer to fully leverage multiple processors on a given machine. It runs on both Unix and Windows.

## The right way: use `Pool`

In [33]:
from multiprocessing import Pool

def use_pool():
    p = Pool(num_threads)
    pwords = dict()
    for v in filter(lambda x: x is not None, 
                    p.map(hash_from_serial, (i for i in range(num_passwords)), chunksize=chunksize)):
        pwords.update(v)
    print(pwords)

Wow! 3x speedup!

In [34]:
%timeit -n 1 -r 1 use_pool()

{'3a7bd3e2360a3d29eea436fcfb7e44c735d117c42d1c1835420b6b9942dd4f1b': 'apple', '74e1bb62f8dabb8125a58852b63bdf6eaef667cb56ac7f7cdba6d7305c50a22f': 'mmmmm', '1115dd800feaacefdf481f1f9070374a2a81e27880f187396db67958b207cbad': 'zyzzx'}
1 loop, best of 1: 19.2 s per loop


# Conclusions

- Use `multiprocessing` not `threading`
- Use `Pool` not `ThreadPool`